In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:

# provide initial link to the any page
loc = 'http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2007/order/true'
html_text = requests.get(loc).text
soup = BeautifulSoup(html_text, 'lxml')


# create empty lists to hold the year and link of the page
year = []
link = []

# From the initial page (can be any year) select <form> tag with js-goto class
# this will have all the links and years, however there is only 1 item in this object
year_link = soup.find_all("form", class_='js-goto')

# loop throuhg the 1 item in the year_link object
# inside are all the links and years under <option> tag
# the attribute of the <option> tag will have the link (minus the first 2 characters)
# the text of the <option> tag will have the year 
# use this link to cycle through all the years
for i in year_link:
    for option in i.find_all('option'):
        year.append(option.text)
        link_str = list(option.attrs.items())[0][1]
        link.append('http://' + link_str[2:len(link_str)]) # remove first 2 characters, slashes
year_link_df = pd.DataFrame({'year':year,'link':link})

In [4]:
# Create function to get player details from each year
def player_details(player_chunk,year):
    """
    player_chunk: supply bs4 ResultSet that has <td> tags for each player
    year: integer value for the year
    returns Pandas df
    """
    rank =[]
    name =[]
    position = []
    height = []
    weight = []
    grade = []
    for profile in player_chunk:
        tds = profile.find_all("td") ###
        rank.append(tds[0].text)
        name.append(tds[1].find('div', class_ = 'name').strong.text)
        position.append(tds[2].text)
        height.append(tds[4].text)
        weight.append(tds[5].text)
        grade.append(tds[7].text)

    return pd.DataFrame({'rank':rank, 'name':name,'position':position,'height':height,'weight':weight,'grade':grade,'year':year})    




#create empty DF
hs_ranking = pd.DataFrame(columns = ['rank', 'name','position','height','weight','grade','year'])

# cycle through each year of the year_link_df and extract player details
for i in range(len(year_link_df)):
    year,link = year_link_df.iloc[i]
    print(year,link)
    html_text = requests.get(link).text
    soup = BeautifulSoup(html_text, 'lxml')
    player_chunk = soup.find_all("tr", class_=re.compile("^odd|^even"))
    hs_ranking = pd.concat([hs_ranking,player_details(player_chunk,year)],axis = 0)

2025 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2025/order/true
2024 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2024/order/true
2023 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/order/true
2022 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2022/order/true
2021 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2021/order/true
2020 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2020/order/true
2019 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2019/order/true
2018 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2018/order/true
2017 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2017/order/true
2016 http://www.espn.com/college-sports/basketball/recruiting/playerrankings/_/class/2016/order/true
2

In [6]:
hs_ranking.to_csv('hs_ranking.csv',index = False)